In [2]:
import pandas as pd
import psycopg2
import numpy as np



from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service



from tqdm import tqdm
tqdm.pandas()
import time



from bs4 import BeautifulSoup
import re

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders


In [3]:
s = Service("../chromedriver_win32/chromedriver.exe")

In [4]:
class postgres_conn:
    def getConn(self):
        try:
            connection = psycopg2.connect(user='postgres',
                                          password='postgres_007',
                                          host="1.pgsql.db.1digitalstack.com",
                                          port='5432',
                                          database='postgres')

            cursor = connection.cursor()
            # Print PostgreSQL Connection properties
            print(connection.get_dsn_parameters(), "\n")

            # Print PostgreSQL version
            return cursor, connection

        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
            return error, error
    def getConn2(self):
        try:
            connection = psycopg2.connect(user='postgres',
                                          password='reporting_007',
                                          host="2.pgsql.db.1digitalstack.com",
                                          port='5432',
                                          database='postgres')

            cursor = connection.cursor()
            # Print PostgreSQL Connection properties
            print(connection.get_dsn_parameters(), "\n")

            # Print PostgreSQL version
            return cursor, connection

        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
            return error, error


    def close_connection(self, cursor, connection):
        # closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")
            
pg = postgres_conn()
conn = pg.getConn()
conn2 = pg.getConn2()

{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': '1.pgsql.db.1digitalstack.com', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': '2.pgsql.db.1digitalstack.com', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [5]:
job_id=input("Enter keystone ids with comma.").split(",")
days=input(" Enter no of days for which data needs brand mapping.\n")
job_id.append("")
tuple(job_id)

Enter keystone ids with comma.64ccf0866364abab2c73bc8e
 Enter no of days for which data needs brand mapping.
2


('64ccf0866364abab2c73bc8e', '')

In [6]:
query = f"""
select brand,title,sku,job_name,channel_id
from ready.serp_details smd join ready.serp_mongo_details m on m.serp_mongo_id=smd.serp_mongo_id
where smd.crawl_date > now() - interval '{days} day' and keystone_job_id in {tuple(job_id)}
"""
serp_details = pd.read_sql_query(query, conn[1])

C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_151972\2162843895.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  serp_details = pd.read_sql_query(query, conn[1])


In [7]:
serp_details

,brand,title,sku,job_name,channel_id
0,ASUS,"ASUS VivoBook P-Series, fully installed and ac...",114062403,Mediamarket r.no.563 Serp_job,58
1,LENOVO,"LENOVO IdeaPad 1i, incl. 1 year Microsoft 365 ...",2776176,Mediamarket r.no.563 Serp_job,58
2,APPLE,"APPLE MacBook Air (2022), MLXW3D/A, notebook w...",2813298,Mediamarket r.no.563 Serp_job,58
3,APPLE,"APPLE MacBook Air (2020), notebook with 13.3 i...",2701413,Mediamarket r.no.563 Serp_job,58
4,APPLE,"APPLE MacBook Air (2020), notebook with 13.3 i...",2701414,Mediamarket r.no.563 Serp_job,58
...,...,...,...,...,...
1155,HP,"HP 250-G8, Core i5, Windows 11 Pro + Office 20...",109350836,Mediamarket r.no.563 Serp_job,58
1156,HP,"HP 17-cp3166ng, Notebook with 17.3 inch displa...",110444633,Mediamarket r.no.563 Serp_job,58
1157,HP,"HP 17CN, Quad Core, Windows 11 Pro + Office 20...",112713418,Mediamarket r.no.563 Serp_job,58
1158,HP,"HP 250-G8, Core i5, Windows 11 Pro + Office 20...",109350901,Mediamarket r.no.563 Serp_job,58


In [20]:
rawdf1=serp_details.drop_duplicates('sku')
rawdf1=rawdf1.reset_index()
df_ASIN = rawdf1[["sku"]]
df_ASIN.dropna(inplace=True)
df_ASIN.drop_duplicates(inplace=True)
asinlist = list(df_ASIN['sku'])
len(asinlist)

     index brand                                              title      sku  \
694   1138    HP  HP 17-cn2356ng, Notebook with 17.3 inch Displa...  2868831   
695   1142    HP  HP Laptop 15s-fq5333ng, notebook with 15.6 inc...  2868907   
696   1144    HP  HP 15a-na0312ng, 15.6-inch Chromebook, Intel® ...  2868902   
697   1145    HP  HP x360 14a-ca0312ng, Chromebook with 14 inch ...  2868854   
698   1147    HP  HP Pavilion 15-eg1376ng, Notebook with 15.6 in...  2790426   

                           job_name  channel_id  
694  Mediamarket r.no.563 Serp_job           58  
695  Mediamarket r.no.563 Serp_job           58  
696  Mediamarket r.no.563 Serp_job           58  
697  Mediamarket r.no.563 Serp_job           58  
698  Mediamarket r.no.563 Serp_job           58  


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_151972\1077363369.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ASIN.dropna(inplace=True)
C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_151972\1077363369.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ASIN.drop_duplicates(inplace=True)


In [21]:
query_2 = f""" 
Select channel_sku_id,country_code, brand_name, categories,category_id
from entity.product_master
where channel_sku_id in {tuple(asinlist)} 
"""
pm_data = pd.read_sql_query(query_2, conn[1])
pm_data

C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_151972\3364747598.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pm_data = pd.read_sql_query(query_2, conn[1])


AttributeError: 'DataFrame' object has no attribute 'duplicates'

In [13]:
mapped_df=pd.merge(rawdf1,pm_data, how='left', left_on=['sku'],right_on=['channel_sku_id'])
mapped_df

,brand,title,sku,job_name,channel_id,channel_sku_id,country_code,brand_name,categories,category_id
0,ASUS,"ASUS VivoBook P-Series, fully installed and ac...",114062403,Mediamarket r.no.563 Serp_job,58,114062403,DE,ASUS,None,None
1,LENOVO,"LENOVO IdeaPad 1i, incl. 1 year Microsoft 365 ...",2776176,Mediamarket r.no.563 Serp_job,58,2776176,DE,LENOVO,None,None
2,APPLE,"APPLE MacBook Air (2022), MLXW3D/A, notebook w...",2813298,Mediamarket r.no.563 Serp_job,58,2813298,DE,APPLE,None,None
3,APPLE,"APPLE MacBook Air (2020), notebook with 13.3 i...",2701413,Mediamarket r.no.563 Serp_job,58,2701413,DE,APPLE,None,None
4,APPLE,"APPLE MacBook Air (2020), notebook with 13.3 i...",2701414,Mediamarket r.no.563 Serp_job,58,2701414,DE,APPLE,None,None
...,...,...,...,...,...,...,...,...,...,...
699,HP,"HP Laptop 15s-fq5333ng, notebook with 15.6 inc...",2868907,Mediamarket r.no.563 Serp_job,58,2868907,DE,HP,None,None
700,HP,"HP 15a-na0312ng, 15.6-inch Chromebook, Intel® ...",2868902,Mediamarket r.no.563 Serp_job,58,2868902,DE,HP,None,None
701,HP,"HP x360 14a-ca0312ng, Chromebook with 14 inch ...",2868854,Mediamarket r.no.563 Serp_job,58,2868854,DE,HP,None,None
702,HP,"HP Pavilion 15-eg1376ng, Notebook with 15.6 in...",2790426,Mediamarket r.no.563 Serp_job,58,2790426,DE,HP,None,None


In [49]:
df2=mapped_df[mapped_df['categories'].isnull()][['channel_id','sku','title','brand_name']]
df2

,channel_id,sku,title,brand_name
0,58,2835388,ASUS ROG Strix X670E-I Gaming WIFI Motherboard...,ASUS
1,58,107478170,"INTEL NUC Mini-PC, Windows 11 Pro, Mini-PC wit...",INTEL
2,58,114100818,"LENOVO Intel N4500 | Windows 11 Pro, Notebook ...",LENOVO
3,58,107477993,"INTEL NUC Mini-PC, Windows 11 Pro, Mini-PC wit...",INTEL
4,58,108089759,"LENOVO Intel U300 5x4.40GHz | Win 11 Pro, note...",LENOVO
...,...,...,...,...
743,58,91653092,"MEGAPORT Gaming PC Intel Core i9 11900KF, Wind...",MEGAPORT
744,58,101593947,"SEDATECH Intel i9-13900KF, Windows 11 Home mul...",SEDATECH
745,58,109793793,SYSTEMTREFF High-End Gaming Intel Core i9-1190...,SYSTEMTREFF
746,58,109794182,"SYSTEMTREFF Pro Gaming Intel Core i9-12900K, W...",SYSTEMTREFF


In [50]:
df=pd.DataFrame(columns=['channel_sku_id','categories','seller_info','first_image'])
df

,channel_sku_id,categories,seller_info,first_image


In [51]:
asinlist = list(df2['sku'])
len(asinlist)

695

In [ ]:
request_no=input("Enter the request number of the requests seperated with underscore(_) ")
#547_552
file='final_output_mediamarket_germany_{}.xlsx'.format(request_no)

In [52]:
chrome_opt=Options()
chrome_opt.add_argument("--incognito")
driver = webdriver.Chrome(service=s,options=chrome_opt)
capabilities = driver.capabilities
chrome_driver_version = capabilities['chrome']['chromedriverVersion']

# Print the ChromeDriver version
print("ChromeDriver version:", chrome_driver_version)

ChromeDriver version: 114.0.5735.90 (386bc09e8f4f2e025eddae123f36f6263096ae49-refs/branch-heads/5735@{#1052})


In [53]:
for i in tqdm(range(0,len(asinlist))):
    sku=asinlist[i]
    url="https://www.mediamarkt.de/de/product/-{}.html".format(sku)
    
    print('************************')
    print(i)
    print(url)
    driver.get(url)
    first_image=''
    bs=BeautifulSoup(driver.page_source,'html.parser')
    cat=[]
    try:
        element = driver.find_elements(By.XPATH, "//ol[@class='Breadcrumb-styled__StyledList-sc-57676b39-1 cXCHDb']//li")
        cat=[element[e].text for e in range(1,len(element))] 
        print(cat)
    except:
         print('cat not  found')
    
    try:
        element = driver.find_element(By.XPATH, "//a[@class='Link-styled__StyledLinkRouter-sc-db43135e-1 ccrLTq']")
        seller_info=element.text.strip()
        print(seller_info)
    except:
        print('seller not found')
    
    try:
        image = bs.find('li',class_="sc-hAQmFe iGozss")
        first_image=image.find('img').get('src')
        print(first_image)
    except:
        print('image not found')
    
        
    df1={'channel_sku_id': sku, 'categories': cat, 'first_image':first_image, 'seller_info':seller_info}
    df=df.append(df1, ignore_index = True)

  0%|                                                                                          | 0/695 [00:00<?, ?it/s]

************************
0
https://www.mediamarkt.de/de/product/-2835388.html


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_148632\3035623616.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df1, ignore_index = True)
  0%|                                                                                | 1/695 [00:08<1:34:59,  8.21s/it]

['Computer & Büro', 'Komponenten', 'Mainboards', 'Mainboard AMD Sockel AM5']
seller not found
https://assets.mmsrg.com/isr/166325/c1/-/ASSET_MMS_97962596?x=480&y=334&format=jpg&quality=80&sp=yes&strip=yes&trim&ex=480&ey=334&align=center&resizesource&unsharp=1.5x1+0.7+0.02&cox=0&coy=0&cdx=480&cdy=334
************************
1
https://www.mediamarkt.de/de/product/-107478170.html


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_148632\3035623616.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df1, ignore_index = True)
  0%|▏                                                                               | 2/695 [00:13<1:15:03,  6.50s/it]

...

['Computer & Büro', 'Notebooks & Zubehör', 'Gaming-Notebooks']
seller not found
image not found
************************
191
https://www.mediamarkt.de/de/product/-103753396.html


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_148632\3035623616.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df1, ignore_index = True)
 28%|█████████████████████▌                                                        | 192/695 [32:49<1:45:04, 12.53s/it]

['Computer & Büro', 'Notebooks & Zubehör', 'Notebooks']
tease solutions GmbH
image not found
************************
192
https://www.mediamarkt.de/de/product/-2864610.html


 28%|█████████████████████▌                                                        | 192/695 [33:01<1:26:30, 10.32s/it]


KeyboardInterrupt: 

In [54]:
df

,channel_sku_id,categories,seller_info,first_image
0,2835388,"[Computer & Büro, Komponenten, Mainboards, Mai...",tease solutions GmbH,https://assets.mmsrg.com/isr/166325/c1/-/ASSET...
1,107478170,"[Computer & Büro, PCs & Zubehör, Mini PCs]",Cybernerds,https://assets.mmsrg.com/isr/166325/c1/-/ASSET...
2,114100818,"[Computer & Büro, Notebooks & Zubehör, Notebooks]",Cybernerds,
3,107477993,"[Computer & Büro, PCs & Zubehör, Mini PCs]",Cybernerds,
4,108089759,"[Computer & Büro, Notebooks & Zubehör, Notebooks]",Cybernerds,https://assets.mmsrg.com/isr/166325/c1/-/ASSET...
...,...,...,...,...
187,2826629,"[Computer & Büro, Notebooks & Zubehör, Gaming-...",IT-tradeport GmbH,https://assets.mmsrg.com/isr/166325/c1/-/ASSET...
188,2804711,"[Computer & Büro, Notebooks & Zubehör, Gaming-...",IT-tradeport GmbH,https://assets.mmsrg.com/isr/166325/c1/-/ASSET...
189,2865276,"[Computer & Büro, Notebooks & Zubehör, Gaming-...",IT-tradeport GmbH,https://assets.mmsrg.com/isr/166325/c1/-/ASSET...
190,2834807,"[Computer & Büro, Notebooks & Zubehör, Gaming-...",IT-tradeport GmbH,


In [55]:
final1=pd.merge(rawdf1,df,left_on='sku', right_on='channel_sku_id', how= 'right')
final=final1[['sku','title','brand','categories','first_image']]

In [56]:
final

,sku,title,brand,categories,first_image
0,2835388,ASUS ROG Strix X670E-I Gaming WIFI Motherboard...,ASUS,"[Computer & Büro, Komponenten, Mainboards, Mai...",https://assets.mmsrg.com/isr/166325/c1/-/ASSET...
1,107478170,"INTEL NUC Mini-PC, Windows 11 Pro, Mini-PC wit...",INTEL,"[Computer & Büro, PCs & Zubehör, Mini PCs]",https://assets.mmsrg.com/isr/166325/c1/-/ASSET...
2,114100818,"LENOVO Intel N4500 | Windows 11 Pro, Notebook ...",LENOVO,"[Computer & Büro, Notebooks & Zubehör, Notebooks]",
3,107477993,"INTEL NUC Mini-PC, Windows 11 Pro, Mini-PC wit...",INTEL,"[Computer & Büro, PCs & Zubehör, Mini PCs]",
4,108089759,"LENOVO Intel U300 5x4.40GHz | Win 11 Pro, note...",LENOVO,"[Computer & Büro, Notebooks & Zubehör, Notebooks]",https://assets.mmsrg.com/isr/166325/c1/-/ASSET...
...,...,...,...,...,...
187,2826629,"LENOVO Legion 5i, gaming notebook with 17.3 in...",LENOVO,"[Computer & Büro, Notebooks & Zubehör, Gaming-...",https://assets.mmsrg.com/isr/166325/c1/-/ASSET...
188,2804711,"LENOVO Legion 5i Pro, premium gaming notebook ...",LENOVO,"[Computer & Büro, Notebooks & Zubehör, Gaming-...",https://assets.mmsrg.com/isr/166325/c1/-/ASSET...
189,2865276,"LENOVO Legion Pro 5, gaming notebook with 16 i...",LENOVO,"[Computer & Büro, Notebooks & Zubehör, Gaming-...",https://assets.mmsrg.com/isr/166325/c1/-/ASSET...
190,2834807,"LENOVO Legion 5i, white backlit keyboard, gami...",LENOVO,"[Computer & Büro, Notebooks & Zubehör, Gaming-...",


In [ ]:
final.to_excel(file)

In [ ]:
email_user = 'alerts@1digitalstack.ai'
email_password = ''
email_send = ['ghansham.pindoriya@1digitalstack.ai','karan.kumar@1digitalstack.ai','arihant.singh@1digitalstack.ai','sambhav.jain@1digitalstack.ai','ankur.dwivedi@1digitalstack.ai','kinshu@1digitalstack.ai']

subject = file
msg = MIMEMultipart()
msg['From'] = email_user
#msg['From'] = "'Abhishek' <{email_user}>"
msg['To'] = ', '.join(email_send)
msg['Subject'] = subject

body = 'Hi,\n\nPFA of today''s mapping request.Please reply as done as you pick this job.'
msg.attach(MIMEText(body,'plain'))

filename=[file]

for file in filename:
    attachment  =open(file,'rb')
    part = MIMEBase('application','octet-stream')
    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition',"attachment; filename= "+file)
    msg.attach(part)
    text = msg.as_string()

server = smtplib.SMTP('1digitalstack.com',25)
#server.login(email_user,email_password)

server.sendmail(email_user,email_send,text)
server.quit()